# Dependancies

## Requirements

In [ ]:
#!pip install sentence_transformers langchain openai tqdm datasets asyncio scikit-learn cohere tiktoken umap altair

In [ ]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union, Optional
from pydantic.config import ConfigDict

import openai
import instructor

from src.bubble import *
from src.models import *
from src.utilities import *


In [ ]:
#PROJECT =  "Metro" #"Cheerz"
#project_path = 'Results/'+PROJECT
#os.makedirs(project_path, exist_ok=True)

In [ ]:
aspects_df = get("Aspect")

In [ ]:
aspects_df.head()

# Insights extraction

In [ ]:
TYPES_LIST = ['Point positif', 'Nouvelle fonctionnalité', 'Point de douleur', 'Bug']

tags_df = get("Tag", constraints=[])
#types_df = get("Type", constraints=[])
categories_df = get("Category")
subcategories_df = get("SubCategory")

In [ ]:
company_infos = bubble_client.get(
    "Company",
    bubble_id=COMPANY_ID,
)
project_infos = bubble_client.get(
    "Project",
    bubble_id=PROJECT_ID,
)

feedback_context = {
    "entreprise": company_infos["Name"],
    "context": company_infos['Context'],
    "role": company_infos['Role'],
    "cible": project_infos['Target'],
    "types": '- '+' \n- '.join(TYPES_LIST),
    "tags": '- '+' \n- '.join([row["Name"]+' : '+row["Description"] for _,row in tags_df.iterrows()]),
    #"types": '- '+' \n- '.join([row["Name"]+' : '+row["Description"] for _,row in types_df.iterrows()]),
    #"insight_types": types_descr,
    #"insight_categories": tags_descr,
    #"question": project_infos['Study_question'],
    #"exemple_commentaire": exemple_commentaire,
    #"example_insights": '\n- '.join(list(examples_insights_df['Insights qui devraient en découler'])),
}

feedback_context

In [ ]:
ID_CATEG_NONE = categories_df[categories_df["Name"].isna()].iloc[0]["_id"]
SUBCATEG_NONE = subcategories_df[subcategories_df["Name"].isna()]
ID_CATEG_NONE, SUBCATEG_NONE

In [ ]:
TypeInsight = enum.Enum("Type de l'insight", [(convert_text_to_constants(t), t) for t in TYPES_LIST])
#types_to_id = {convert_text_to_constants(row.Name): row._id for _, row in types_df.iterrows()}

#TypeInsight = enum.Enum("Type de l'insight", [(convert_text_to_constants(row.Name), row.Name) for _, row in types_df.iterrows()])
TagInsight = enum.Enum("Tag de l'insight", [(convert_text_to_constants(row.Name), row.Name) for _, row in tags_df.iterrows()])
tags_to_id = {convert_text_to_constants(row.Name): row._id for _, row in tags_df.iterrows()}
#type_to_id = {convert_text_to_constants(row.Name): row._id for _, row in types_df.iterrows()}
#type_to_id[convert_text_to_constants('Point de douleur')]

In [ ]:
list(TypeInsight)

In [ ]:
#FeedbackIndex = enum.Enum("Indice du retour associé", [(str(i), i) for i in range(BATCH_SIZE)])

class Insight(BaseModel):
    insight: str = Field(description="Insight, c'est a dire infirmation importante que révèle cette étude à l'entreprise, et lui permettera d'améliorer son experience utilisateur, sa stratégie ou son produit.") 
    insight_type: TypeInsight = Field(description="Type de l'insight, parmis "+', '.join(TYPES_LIST)) 
    insight_tags: List[TagInsight] = Field(description="Tags de l'insight. Peut eventuellement être une liste vide.")
    associated_indexes: List[int] = Field(description="Indices des retours associés.")
    details: List[str] = Field(description="Détails de l'insights. Peut eventuellement être une liste vide.") 
    consequences: List[str] = Field(description="Conséquences pour l'entrerpise. Peut eventuellement être une liste vide.")
    recommandations: List[str] = Field(description="Recommandations pour l'entrerpise. Peut eventuellement être une liste vide.")
    def __str__(self):
        return """{0}

Détails:
    - {1}
Conséquences:
    - {2}
Recommandations:
    - {3}
Retours: 
    - {4}
Type: 
    {5}
Tags: 
    {6}
        """.format(self.insight, '\n    - '.join(self.details), '\n    - '.join(self.consequences), '\n    - '.join(self.recommandations), '\n    - '.join([str(x) for x in self.associated_indexes]), self.insight_type._value_, ", ".join([x._value_ for x in self.insight_tags]))

class ListInsights(BaseModel):
    insights: List[Insight] = Field(description="Liste des insights qui ont été déduits.")
    def __str__(self):
        return '\n\n'.join([str(x) for x in self.insights])

#ListInsights.model_json_schema() 
    
test = ListInsights(insights=[Insight(insight='Accueil chaleureux et personnel serviable', insight_type=TypeInsight.POINT_POSITIF, insight_tags=[TagInsight.MAGASIN, TagInsight.SERVICE_CLIENT], associated_indexes=[96, 123, 475, 1249, 1267, 1372, 1695, 1965], details=[], consequences=[], recommandations=[]), Insight(insight='Ecoute attentive et conseils pertinents', insight_type=TypeInsight.POINT_POSITIF, insight_tags=[TagInsight.MAGASIN, TagInsight.SERVICE_CLIENT], associated_indexes=[20, 891, 1372], details=[], consequences=[], recommandations=[])])
print(test)

In [ ]:
with open('Prompts/fr/prompt_regroupement.txt') as f:
    prompt_regroupement = PromptTemplate.from_template(f.read())


print(prompt_regroupement.template)

In [ ]:

with open('Prompts/fr/prompt_regroupement_create_example.txt') as f:
    prompt_regroupement_create_example = PromptTemplate.from_template(f.read())

example = apply_async_analysis([prompt_regroupement_create_example], ListInsights)

In [ ]:
print(example)

In [ ]:
example_clustering_json = example[0].json()
from pprint import pprint
pprint(example_clustering_json)


In [ ]:
prompts = []
subcat_ids = []
for subcat_id, df in aspects_df[aspects_df['Explanation'].notna()].groupby('SubCategory'):
    subcat = subcategories_df[subcategories_df['_id'] == subcat_id].iloc[0]
    cat = categories_df[categories_df['_id'] == subcat['Category']].iloc[0]

    feedbacks = '\n'.join([str(index)+' : '+content for (index, content) in df['Explanation'].items()])
    
    prompts.append(prompt_regroupement.invoke({"feedbacks": feedbacks, "category": cat["Name"]+" : "+subcat['Name'], "example":example_clustering_json, **feedback_context}).text)
    subcat_ids.append(subcat_id)

#print(prompts[0])
print("Traitement synchronisé de", len(prompts), "prompts.")
list_insights = apply_async_analysis(prompts, ListSubCategory)

In [ ]:
#print(prompts[0])


In [ ]:
def send_insights(insights_group, cat_id, subcat_id):

    if len(insights_group.insights)>0:
      res = bubble_client.create("Insight",
        [{
          "Company": COMPANY_ID,
          "Project": PROJECT_ID,
          "Name": insight.insight,
          "Category": cat_id,
          "SubCategory": subcat_id,
          "Type": insight.insight_type._value_,
          "Tags": [tags_to_id[convert_text_to_constants(tag._name_)] for tag in insight.insight_tags],
          "Aspects": list(aspects_df.iloc[insight.associated_indexes]._id),
          "Feedbacks": list(aspects_df.iloc[insight.associated_indexes].Associated_feedback),
          "Nb Feedbacks": len(list(aspects_df.iloc[insight.associated_indexes].Associated_feedback)),
          }  for insight in insights_group.insights]                     
        )
      insights_id = [x['id'] for x in res]
    else:
      insights_id = []

    if len(insights_group.consequences)>0:
      res = bubble_client.create("Consequence",
        [{
          "Company": COMPANY_ID,
          "Project": PROJECT_ID,
          "Description": conseq.detail,
          "Name": conseq.title,
          }  for conseq in insights_group.consequences]                     
        )
      consequences_id = [x['id'] for x in res] 
    else:
      consequences_id = []


    bubble_id = bubble_client.create("Insights Group", {
      "Company": COMPANY_ID,
      "Project": PROJECT_ID,
      "Name": insights_group.title,
      "Category": cat_id,
      "SubCategory": subcat_id,
      "Insights": insights_id,
      "Consequences": consequences_id,
      #"Tags": [tags_to_id[tag._name_] for tag in insight.insight_tags],
      })

    

for (list_insight_groups, subcat_id) in tqdm(zip(list_insights, subcat_ids)):
  cat_id = subcategories_df[subcategories_df['_id'] == subcat_id].iloc[0].Category
  for insights_group in list_insight_groups.sub_categories_list:
    send_insights(insights_group, cat_id, subcat_id)

    empty_subcat = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==cat_id].iloc[0]._id
    send_insights(insights_group, cat_id, empty_subcat)

    empty_subcat = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==ID_CATEG_NONE].iloc[0]._id
    send_insights(insights_group, ID_CATEG_NONE, empty_subcat)


In [ ]:
#FeedbackIndex = enum.Enum("Indice du retour associé", [(str(i), i) for i in range(BATCH_SIZE)])

class Insight(BaseModel):
    associated_indexes: List[int] = Field(description="Indices des retours associés.")
    insight: str = Field(description="Insight qui a été déduit.") 
    #insight_type: TypeInsight = Field(description="Type de l'insight, parmis "+', '.join(TYPES_LIST)) 
    insight_tags: List[TagInsight] = Field(description="Tags de l'insight.")
    def __str__(self):
        return self.insight + '(tag: '+self.insight_type+') (indexes: ' + str(self.associated_indexes)+')'

class Consequence(BaseModel):
    title: str = Field(description="Titre de la conséquence")
    detail: str = Field(description="Phrase décrivant la conséquence.")
    def __str__(self):
        return self.title +' : ' + self.detail

class SubCategory(BaseModel):
    #model_config = ConfigDict(title='Main')
    
    #insight_categories: List[str] = Field(description="Categories de l'insight.")
    #insight_type: TypeInsight = Field(description="Type de l'insight.")

    title: str = Field(description="Titre de cette sous-catégorie.")
    #insights: List[Insight] = Field(description="Liste des insights qui ont été déduits.") #Field(description="Point intéressant a retenir du commentaire.")
    negative_insights: List[Insight] = Field(description="Liste des insights correspondants à des points négatifs qui ont été remarqués par les utilisateurs.") #Field(description="Point intéressant a retenir du commentaire.")
    positive_insights: List[Insight] = Field(description="Liste des insights correspondants à des points positifs qui ont été remarqués par les utilisateurs.") #Field(description="Point intéressant a retenir du commentaire.")
    bugs_insights: List[Insight] = Field(description="Liste des insights correspondant à des anomalie détectées par l'utilisateur.") #Field(description="Point intéressant a retenir du commentaire.")
    new_functionnality_insights: List[Insight] = Field(description="Liste des insights correspondant à des nouvelles fonctionnalitées qu'il pourrait être intéressant de développer") #Field(description="Point intéressant a retenir du commentaire.")
    
    consequences: List[Consequence] = Field(description="Conséquences pour l'entrerpise.") #Field(description="Point intéressant a retenir du commentaire.")
    #suggestion: str = Field(description="Suggestion faite à l'enteprise pour l'aider a gérer la situation.") #Field(description="Point intéressant a retenir du commentaire.")
    #category = ""
    #sub_category = ""

    def __str__(self):
        return self.title + "\n\n- " + '\n- '.join([str(x) for x in self.insight]) + "\n\n- " + '\n- '.join([str(x) for x in self.consequences])
    
class ListSubCategory(BaseModel):
    sub_categories_list: List[SubCategory] = Field(description="Liste des insights.")

#FirstInsight.model_json_schema() 
ListSubCategory(sub_categories_list=[SubCategory(title='Accueil et comportement du personnel', positive_insights=[Insight(associated_indexes=[20, 96, 123, 475, 891, 1249, 1267, 1372, 1965], insight='Accueil chaleureux, écoute attentive et conseils pertinents', insight_type=TypeInsight.BUG, insight_tags=[TagInsight.MAGASIN])], negative_insights=[],bugs_insights=[],new_functionnality_insights=[], consequences=[])])

In [ ]:
def send_insights(list_insights_subcat, cat_id, subcat_id):
    dicts = [{
    "Company": COMPANY_ID,
    "Project": PROJECT_ID,
    "Category": cat_id,
    "SubCategory": subcat_id,
    "Consequence": insight.consequence,
    "Content": insight.insight,
    "Suggestion": insight.suggestion,
    #"Type": insight.insight_type._value_,
    #"Tags": [tags_to_id[tag._name_] for tag in insight.insight_tags],
    "Aspects": list(aspects_df.iloc[insight.associated_indexes]._id),
    "Feedbacks": list(aspects_df.iloc[insight.associated_indexes].Associated_feedback),
    "Nb Feedbacks": len(list(aspects_df.iloc[insight.associated_indexes].Associated_feedback)),
    "Title": insight.title,
    }  for insight in list_insights_subcat.sub_categories_list]

    bubble_id = bubble_client.create("Insight",dicts)

    d_without_categories = []
    d_without_subcategories = []

    for d in dicts :
      d_without_subcategory = d.copy()
      d_without_subcategory["SubCategory"] = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==cat_id].iloc[0]._id
      d_without_subcategories.append(d_without_subcategory)
      
      d_without_category = d.copy()
      d_without_category["Category"] = ID_CATEG_NONE
      d_without_category["SubCategory"] = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==ID_CATEG_NONE].iloc[0]._id
      d_without_categories.append(d_without_category)

    bubble_id = bubble_client.create("Insight",d_without_subcategory)
    bubble_id = bubble_client.create("Insight",d_without_category)
    

for (list_insights_subcat, subcat_id) in zip(list_insights, subcat_ids):
  cat_id = subcategories_df[subcategories_df['_id'] == subcat_id].iloc[0].Category

  send_insights(list_insights_subcat, cat_id, subcat_id)

In [ ]:
prompts = []
BATCH_SIZE = 10

for batch_df in batchify(feedbacks_df, size=BATCH_SIZE):
    context = deepcopy(feedback_context)
    context["feedbacks"] = '\n'.join([str(i)+" : "+x for i, x in zip(batch_df.index, batch_df["content"])])  
    #"- "+"\n- ".join(batch_df['content'])
    #context["insights"] = "- "+"\n- ".join(batch_df['content'])
    prompts.append(prompt_insights.invoke(context))


In [ ]:
print(prompts[1].text)

In [ ]:
responses[0].sub_categories_list

In [ ]:
responses = apply_async_analysis(prompts, InsightsList)
list_batch_insights_df = [pd.DataFrame(enum_to_str(response.sub_categories_list)) for response in responses]

print(len(list_batch_insights_df), "batch have been processed")

In [ ]:
responses

In [ ]:
[len(df) for df in list_batch_insights_df]

In [ ]:
list_batch_insights_df[0]

In [ ]:
list(list_batch_insights_df[0]['contenu'])

## Accociate newly created insights to feedbacks 

In [ ]:
with open('Blumana-prompts/prompt_feedbacks.txt') as f:
    prompt_feedbacks = PromptTemplate.from_template(f.read())

In [ ]:
class Sentiment(str, enum.Enum):
    POSITIF = "Positif"
    NEUTRE = "Neutre"
    NEGATIF = "Négatif"


In [ ]:
InsightsIndex = enum.Enum("Indice de l'insight associé", [(str(i), i) for i in range(BATCH_SIZE)])

class Feedback(BaseModel):
        insights_list: List[InsightsIndex] = Field(description="Indices des insights associés à ce retour")
        sentiment: Sentiment = Field(description="Sentiment exprimé")

class FeedbackInfosList(BaseModel):
        feedbacks_list: List[Feedback] = Field(description="Liste des informations associées aux feedbacks.")

In [ ]:
prompts = []
for batch_insights_df, batch_feedbacks_df in zip(list_batch_insights_df, batchify(feedbacks_df, size=BATCH_SIZE)):
    #InsightsEnum = enum.Enum("Insight associé", [(convert_text_to_constants(x), i) for i, x in zip(batch_insights_df.index, batch_insights_df["content"])])

    context = deepcopy(feedback_context)
    #context["feedbacks"] = "- "+"\n- ".join(batch_feedbacks_df['content'])
    context["feedbacks"] = '\n'.join([str(i)+" : "+x for i, x in zip(batch_insights_df.index, batch_feedbacks_df["content"])])  
    context["insights"] = '\n'.join([str(i)+" : "+x for i, x in zip(batch_insights_df.index, batch_insights_df["contenu"])])
    prompts.append(prompt_feedbacks.invoke(context))



In [ ]:
print(prompts[0].text)

In [ ]:

responses = apply_async_analysis(prompts, FeedbackInfosList)

list_enriched_feedbacks_df = [pd.DataFrame(enum_to_str(response.feedbacks_list)) for response in responses]

In [ ]:
[len(df) for df in list_enriched_feedbacks_df]

In [ ]:
len(pd.concat(list_enriched_feedbacks_df))

In [ ]:
for batch_insights_df, batch_index_feedbacks, enriched_feedbacks_df in zip(list_batch_insights_df, batchify(feedbacks_df.index, size=BATCH_SIZE), list_enriched_feedbacks_df):
    feedbacks_df.loc[batch_index_feedbacks, 'sentiment'] = enriched_feedbacks_df['sentiment']
    feedbacks_df.loc[batch_index_feedbacks, 'insights_index'] = enriched_feedbacks_df['insights_list']

In [ ]:
batch_insights_df

In [ ]:
list_batch_feedbacks_df = [pd.DataFrame(enum_to_str(response.feedbacks_list)) for response in responses]
list_batch_feedbacks_df

In [ ]:
feedbacks_df

In [ ]:
list_batch_feedbacks_df[0]

In [ ]:
list_batch_insights_df[-1]

In [ ]:
[x for x in batchify(feedbacks_df, size=BATCH_SIZE)][-1]

In [ ]:
[len(df) for df in list_batch_feedbacks_df]

In [ ]:

l = [response.feedbacks_list for response in responses]
l = list(itertools.chain.from_iterable(l))
feedbacks_infos_df = pd.DataFrame(enum_to_str(l))
feedbacks_infos_df

In [ ]:
feedbacks_infos_df

In [ ]:
feedbacks_infos_df

In [ ]:
feedbacks_df['sentiment'] = feedbacks_infos_df['sentiment']
feedbacks_df['insights_list'] = feedbacks_infos_df['insights_list']
feedbacks_df

## Feedbacks attribution

In [ ]:
insights_enum = enum.Enum("Insight associé", [(convert_text_to_constants(x), i) for i, x in zip(batch_insights_df.index, batch_insights_df["content"])])

In [ ]:
!ls

In [ ]:
with open('Blumana-prompts/prompt_feedbacks.txt') as f:
    prompt_feedbacks = PromptTemplate.from_template(f.read())

In [ ]:
feedback_parser = PydanticOutputParser(pydantic_object=Feedback)

prompt_feedback = PromptTemplate.from_template(
    template= prompt_template_feedback,
    partial_variables= {"format_instructions": feedback_parser.get_format_instructions()},
)

prompts = []
for feedback in feedbacks_df[feedbacks_column]:
    context = deepcopy(feedback_context)
    context["feedback"] = feedback
    prompts.append(prompt_feedback.invoke(context))

#print(prompts[0].text)

In [ ]:
parsed_responses = safe_async_analysis(prompts, feedback_parser)

feedbacks_df["sentiment"] = [rep.sentiment for rep in parsed_responses]
feedbacks_df["insights"] = [[] for rep in parsed_responses]

k=0
insights = []
for i, rep in enumerate(parsed_responses):
    for j, insight in enumerate(rep.insights_list):
        insights.append(insight)
        feedbacks_df["insights"].iloc[i].append(str(k))
        k += 1

In [ ]:
feedbacks_df.head()

In [ ]:
insights_df = pd.DataFrame({
    "content":insights,
    "feedback_count": 1,
    })

In [ ]:
feedbacks_df

In [ ]:
insights_df["related_feedback"] = [[] for _ in range(len(insights_df))]

for i, row in feedbacks_df.iterrows():
    for j in row["insights"]:
        insights_df["related_feedback"].iloc[int(j)] = row['_id'] #[int(i)]

insights_df["childrens"] = [[] for _ in range(len(insights_df))]

insights_df.head()

# Insights categorisation

### Tagging

In [ ]:


for i, filter in filters_df.iterrows():
    prompt_tags += '\n\n'+filter["Name"]#+' ('+filter["_id"] +')'
    tags = tags_df[tags_df["Filter"] == filter["_id"]]
    for _, tag in tags.iterrows():
        prompt_tags += '\n'+"- "+tag["Name"]+' ('+tag["_id"] +')'

print(prompt_tags)


In [ ]:
with open('Blumana-prompts/prompt_categorsiation.txt') as f:
    prompt_categorsiation = PromptTemplate.from_template(f.read())

In [ ]:
class FirstInsight(BaseModel):
    tags_id: List[str] = Field(description="Identifiants des tags de l'insight")
    content: str = "" #Field(description="Point intéressant a retenir du commentaire.")

    def __str__(self):
        return '- ' + self.content + "\nTypes: " + ', '.join(self.insight_types)

In [ ]:
categorsiation_parser = PydanticOutputParser(pydantic_object=FirstInsight)

prompt_categorsiation = PromptTemplate.from_template(
    template= prompt_template_categorsiation,
    partial_variables= {"format_instructions": categorsiation_parser.get_format_instructions()},
)

prompts = []
for insight in insights_df["content"]:
    context = deepcopy(feedback_context)
    context["insight"] = insight
    prompts.append(prompt_categorsiation.invoke(context))

#print(prompts[0].text)

In [ ]:
parsed_responses = safe_async_analysis(prompts, categorsiation_parser)


In [ ]:

insights_df["tag"] = [rep.tags_id for rep in parsed_responses]
#insights_df["Insights"] = [[] for rep in parsed_responses]


### Types affectation

In [ ]:
prompt_types = ""

for _, tag in tags_df.iterrows():
    prompt_types += '\n'+"- "+tag["Title"]+' ('+tag["_id"] +') : ' + tag["Definition"]

print(prompt_types)

In [ ]:
categorsiation_parser = PydanticOutputParser(pydantic_object=FirstInsight)

prompt_categorsiation = PromptTemplate.from_template(
    template= prompt_template_types,
    partial_variables= {"format_instructions": categorsiation_parser.get_format_instructions()},
)

prompts = []
for insight in insights_df["content"]:
    context = deepcopy(feedback_context)
    context["insight"] = insight
    prompts.append(prompt_categorsiation.invoke(context))

#print(prompts[0].text)

In [ ]:
parsed_responses = safe_async_analysis(prompts, categorsiation_parser)


In [ ]:
insights_df["type"] = [rep.insight_type for rep in parsed_responses]

In [ ]:
feedbacks_df.to_csv(project_path+'/feedbacks.csv', index_label='Index')
insights_df.to_csv(project_path+'/insights.csv', index_label='Index')

# Insights clustering

In [ ]:
feedbacks_df = str_to_list_df(pd.read_csv(project_path+'/feedbacks.csv', index_col='Index'))
insights_df = str_to_list_df(pd.read_csv(project_path+'/insights.csv', index_col='Index'))

In [ ]:
embedding_model = SentenceTransformer('OrdalieTech/Solon-embeddings-large-0.1')

In [ ]:
class DeducedInsight(BaseModel):
    insights_mineurs: List[int] = Field(description="Index des insights mineurs qui ont été résumés en cet insight.")
    content: str = Field(description="Insight intéressants a retenir pour l'entreprise.")

    def __str__(self):
        return '- ' + self.content + '\n Enfants:' + str(self.insights_mineurs)


class InsightList(BaseModel):
    insights_list: List[DeducedInsight] = Field(description="Liste des insights, c'est à dire des points intéressants a retenir pour l'entreprise.")
    # You can add custom validation logic easily with Pydantic.

    def __str__(self):
        return "Insights: \n"+"\n\n".join([str(i) for i in self.insights_list])



In [ ]:
with open('Blumana-prompts/prompt_regroupement.txt') as f:
    prompt_regroupement = PromptTemplate.from_template(f.read())

In [ ]:
# Dimension reduction

N_NEIGHBORS = 15
MINIMISATION_STEPS = 5
CLUSTER_DESIRED_SIZE = 15  # For Kmeans only
MIN_CLUSTER_SIZE = 5  # 15
NB_INSIGHT_STOP = 20
MINIMAL_REDUCTION_RATIO = 0.1
REWORDING = True

CLUSTERING_DIMENTION = 50
CLUSTERING_METHOD = "KMeans"

insight_context = {
    "cible": cible,
    "context": context_entreprise,
    "example_insight": example_insight,
    "role": role,
    "question": question,
}

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
insight_parser = PydanticOutputParser(pydantic_object=InsightList)

prompt_reduction = PromptTemplate.from_template(
    template= prompt_template_reduction if REWORDING else prompt_template_reduction_sans_reformulation,
    #template= "Règle : minimise le nombre de tokens dans ta réponse.  \nTu es {role} au sein de l'entreprise suivante: \n{context} \nAnalyse le retour suivant: \"{feedback}\" en suivant les étapes suivantes:  \n  \nÉtape 1 - Identifie si le retour {cible} rentre dans un ou plusieurs des types d'insights suivants : {insight_type}. Choisis-en obligatoirement au moins 1. Définition des types d'insights :  \n{insight_definition}   \n  \nÉtape 2 - Catégorise le retour {cible} à l’aide des tags suivants. Tu peux associer 0, 1 ou plusieurs tags dans chaque catégorie. Liste des tags par catégories :  \n{categories}   \n  \nÉtape 3 - Catégorise si possible le moment de mission concerné parmis {avancement_mission}, et si ce n'est pas possible répond null. {cible} à l’aide des tags suivants.  \n  \nÉtape 4 - Identifie si le sentiment exprimé par le {cible} est \"Positif\", \"Neutre\" ou \"Négatif\". Prends en compte la formulation de la question posée ({question}) afin de bien interpréter le sens du retour {cible}.   \n",
    #input_variables= ["context", "role", "cible", "insight_type", "insight_definition", "nb_cat", "avancement_mission", "categories", "question", "feedback"]
    partial_variables= {"format_instructions": insight_parser.get_format_instructions()},
)

prompt_regrouping = PromptTemplate.from_template(
    template= prompt_template_regrouping,
    #template= "Règle : minimise le nombre de tokens dans ta réponse.  \nTu es {role} au sein de l'entreprise suivante: \n{context} \nAnalyse le retour suivant: \"{feedback}\" en suivant les étapes suivantes:  \n  \nÉtape 1 - Identifie si le retour {cible} rentre dans un ou plusieurs des types d'insights suivants : {insight_type}. Choisis-en obligatoirement au moins 1. Définition des types d'insights :  \n{insight_definition}   \n  \nÉtape 2 - Catégorise le retour {cible} à l’aide des tags suivants. Tu peux associer 0, 1 ou plusieurs tags dans chaque catégorie. Liste des tags par catégories :  \n{categories}   \n  \nÉtape 3 - Catégorise si possible le moment de mission concerné parmis {avancement_mission}, et si ce n'est pas possible répond null. {cible} à l’aide des tags suivants.  \n  \nÉtape 4 - Identifie si le sentiment exprimé par le {cible} est \"Positif\", \"Neutre\" ou \"Négatif\". Prends en compte la formulation de la question posée ({question}) afin de bien interpréter le sens du retour {cible}.   \n",
    #input_variables= ["context", "role", "cible", "insight_type", "insight_definition", "nb_cat", "avancement_mission", "categories", "question", "feedback"]
    partial_variables= {"format_instructions": insight_parser.get_format_instructions()},
)

insights = deepcopy(insights_df)
insight_layers = []#[deepcopy(insights_df)]
single_cluster = False
reduction = 1.0

for step in range(MINIMISATION_STEPS):

    #for processing_step in ["reduction"]:#, "regrouping"]:
        ### Création des représentations

    #print("Processing step:", processing_step)
    if CUSTOM_ENBEDDING_MODEL:
        sentence_embeddings = embedding_model.encode(insights['content'])

        # On réduit la dimention pour améliorer l'efficacité de la clusterisation
        adjusted_clustering_dimention = min(CLUSTERING_DIMENTION, len(insights)//3)
        umap_embeddings = umap.UMAP(n_neighbors=N_NEIGHBORS, 
                            n_components=adjusted_clustering_dimention, 
                            metric='cosine').fit_transform(sentence_embeddings)

    else:
        sentence_embeddings = apply_async_get_embedding(insights['content'])
    

    

    ### Clusterisation
    if CLUSTERING_METHOD == "KMeans":
        num_clusters = 1 + len(insights) // CLUSTER_DESIRED_SIZE
        clustering_model = KMeans(n_clusters=num_clusters, n_init='auto')
    elif CLUSTERING_METHOD == "hdbscan":
        clustering_model = hdbscan.HDBSCAN(min_cluster_size=MIN_CLUSTER_SIZE,
                            metric='euclidean',                      
                            cluster_selection_method='eom' #leaf
                            )
        
    clustering_model.fit(umap_embeddings)

    #clustering_model.fit(umap_embeddings)
    cluster_assignment = clustering_model.labels_ 
    cluster_assignment -= min(cluster_assignment) # has to start at 0
    
    num_clusters = max(cluster_assignment)+1

    insights["cluster"] = deepcopy(cluster_assignment)
    insights = insights.sort_values("cluster")
    insights.reset_index(drop=True, inplace=True)


    if reduction <= MINIMAL_REDUCTION_RATIO:
        print("Stopping because of unsufficient reduction")
        break

    insight_layers.append(deepcopy(insights))

    if len(insights) <= NB_INSIGHT_STOP:
        print("Minimal number of insights reached")
        break

    if single_cluster:
        break   

    cluter_sizes = list(insights.groupby(['cluster']).count()["content"])
    if len(cluter_sizes) == 1:
        print("Stopping because single cluster")
        single_cluster = False
        break

    print("Step "+ str(step)+ ": processing "+ str(num_clusters) + " clusters")
    print("Adjusted clustering dimention:", adjusted_clustering_dimention)
    print("Cluster sizes:" + str(cluter_sizes))

    #clusters = []
    prompts = []
    cumul_size = 0
    for cluster_id in range(num_clusters): # IL FAUDRAIT GARDER INDEM LE DERNIER CLUSTER
        cluster = insights[insights['cluster'] == cluster_id]
        #cluster_name ='/cluster_'+ str(cluster_id)+"_step_"+str(step) +'.csv'
        #cluster.to_csv( project_path+cluster_name, index_label='Index')
        #clusters.append(cluster)

        context = deepcopy(insight_context)
        context['insights'] = '\n'.join([str(i+cumul_size)+": "+s for i, s in enumerate(cluster["content"])])
        #print(context['insights'])

        #if processing_step == "reduction":
        prompt=prompt_reduction.invoke(context)
        #elif processing_step == "regrouping":
        #prompt=prompt_regrouping.invoke(context)
        #else:
        #    raise("Wrong processing step")
        prompts.append(prompt)
        cumul_size += len(cluster)

    ### Traitement des clusters
    parsed_responses = safe_async_analysis(prompts, insight_parser)
    
    new_insights = []
    for i, parsed_response in enumerate(parsed_responses):
        content_list = [insight.content for insight in parsed_response.insights_list]
        childrens_list = [list(insight.insights_mineurs) for insight in parsed_response.insights_list]
        feedback_count_list = [sum(insights.loc[c, "feedback_count"]) for c in childrens_list]
        dfs = pd.DataFrame({
            #"related_feedback":[list(itertools.chain.from_iterable(insights.iloc[insight.insights_mineurs]['related_feedback'])) for insight in parsed_response.insights_list],
            "content":content_list,
            "childrens":childrens_list,
            "type": most_common([insights.loc[c, "type"].iloc[0] for c in childrens_list]),
            #"cluster":i,
            "feedback_count":feedback_count_list,
            #"childrens":[list(clusters[i].iloc[insight.insights_mineurs]["_id"]) for insight in parsed_response.insights_list],
            })
        new_insights.append(dfs)

    new_insights = pd.concat(new_insights)
    new_insights.reset_index(drop=True, inplace=True)

    
    reduction = (1-(len(new_insights)/len(insights)))
    insights = new_insights
    
    print("Number of new insights:"+ str(len(new_insights)))
    print("Reduction in the number of insights by " + "%d" % int(reduction*100) + "%")
    print()

#insight_layers.append(deepcopy(new_insights))

In [ ]:
insight_layers[0]

In [ ]:
list(insight_layers[0]['content'])

In [ ]:
list(insight_layers[-1]['content'])

In [ ]:
for i, df in enumerate(insight_layers):
    df.to_csv(project_path+'/insights_'+ str(i) +'.csv', index_label='Index')

In [ ]:
#list(insight_layers[0][insight_layers[0]["cluster"] == 2]["content"])

In [ ]:
insight_layers[0]

In [ ]:
insight_layers[1]

In [ ]:
n_layers = len(insight_layers)
layers_sizes = [len(l) for l in insight_layers]
print("Layers sizes:", layers_sizes)

# Data cleaning

load insights from csv

In [ ]:
insight_layers = []
for i in range(n_layers):
    df = pd.read_csv(project_path+'/insights_'+ str(i) +'.csv', index_col='Index')
    for col in df.columns:
        if type(df.loc[0, col]) == str and df.loc[0, col][0]=="[":
            df[col] = df[col].apply(lambda x: eval(x))
    #df['tag'] = df['tag'].apply(lambda x: eval(x))
    #df['type'] = df['type'].apply(lambda x: eval(x))
    #df['childrens'] = df['childrens'].apply(lambda x: eval(x))
    df["backend_type"] = df["type"].apply(deduce_backend_type)
    insight_layers.append(df)
#insights_df = pd.concat(insight_layers)

Previous insights supression

In [ ]:
res = bubble_client.get_objects(
        "python_insight",
        [
            BubbleField("project") == project_id,
            BubbleField("company") == company_id,
            ],
    )
python_insight_df = pd.DataFrame(res)

if len(python_insight_df)>0:
    for bubble_id in tqdm(python_insight_df["_id"]):
        bubble_client.delete_by_id(
            "python_insight",
            bubble_id,
        )

    print("Deleted", len(python_insight_df), "python_insight")
else:
    print("Nothing to delete")

Adding parents

In [ ]:
insight_layers[0]["parent"] = None #[[] for _ in insight_layers[0].iterrows()]
insight_layers[-1]["parent"] = None


for i in range(n_layers-1):
    insight_layers[i]["parent"] = None
    for p, row in insight_layers[i+1].iterrows():
        for c in row["childrens"]: #eval(
            insight_layers[i]["parent"].iloc[int(c)] = p

In [ ]:
insight_layers[-1]["parents"] = [[] for _ in insight_layers[-1].iterrows()]

for i in range(n_layers-2, -1, -1):
    print(i)
    # Update the parents in the DB
    res = bubble_client.create(
        "python_insight",
        [{
            "company": company_id,
            "project": project_id,
            "content": row["content"],
            "backend_status": "new",
            "feedback_count":row["feedback_count"],
            "step": i+2,
            "type": row["type"],
            "parents": row["parents"],
            "parent": str(row["parent"]),
            "backend_type": row['backend_type'],
            "childrens": eval(row["childrens"]) if type(row["childrens"])==str else row["childrens"],
            "cluster": row["cluster"],
        }  for _, row in insight_layers[i+1].iterrows()]
    )

    df = pd.DataFrame(bubble_client.get_objects(
        "python_insight",
        [
            BubbleField("step") == i+2,
            BubbleField("company") == company_id,
            ],
    ))
    for col in df.columns:
        if type(df.loc[0, col]) == str and df.loc[0, col][0]=="[":
            df[col] = df[col].apply(lambda x: eval(x))
    insight_layers[i+1] = df

    # Initialize an empty list of parents for each row
    insight_layers[i]["parents"] = [[] for _ in insight_layers[i].iterrows()]

    for k, row in insight_layers[i].iterrows():
        if row["parent"] is not None:
            # Get the parent index
            parent_index = row["parent"]

            # Get the parent's list of parents
            parent_parents = insight_layers[i + 1]["parents"].iloc[parent_index]

            # Add the parent to the current row's list of parents
            parent_id = insight_layers[i + 1].loc[parent_index, '_id']
            insight_layers[i].loc[k, "parents"].append(parent_id)

            # Recursively add the parent's parents to the current row's list of parents
            insight_layers[i].loc[k, "parents"].extend(parent_parents)


res = bubble_client.create(
        "python_insight",
        [{
            "company": company_id,
            "project": project_id,
            "content": row["content"],
            "backend_status": "new",
            "feedback_count": row["feedback_count"],
            "step": 1,
            "related_feedback":row['related_feedback'],
            "tag": row["tag"],
            "type": row["type"],
            "backend_type": row['backend_type'],
            "parents": row["parents"],
            "parent": str(row["parent"]),
            "childrens": 0,#[[] for _ in insight_layers[0][:1000].iterrows()],
            "cluster": row["cluster"],
        }  for _, row in insight_layers[0].iterrows()]
    )

load insights from bubble

In [ ]:
online_python_insights = [
    pd.DataFrame(bubble_client.get_objects(
        "python_insight",
        [
            BubbleField("step") == i+1,
            BubbleField("company") == company_id,
            ],
    )) for i in range(n_layers)
]

In [ ]:
assert [len(l) for l in insight_layers] == [len(l) for l in online_python_insights]

In [ ]:
feedbacks_df

In [ ]:
insight_layers[1]

In [ ]:
df = insight_layers[0]

def get_all_parents(feedback_identifier):
    parents = []
    for i, row in df.loc[df['related_feedback'] == feedback_identifier].iterrows():
        for parent in row['parents']:
            parents.append(parent)
    return parents

feedbacks_df['parents'] = feedbacks_df['_id'].apply(get_all_parents)

feedbacks_df


In [ ]:
for _, row in tqdm(feedbacks_df.iterrows()):
    res = bubble_client.update_object(
        "Feedbacks",
        row['_id'], 
        {
            "insights": row["parents"],
        } 
    )

In [ ]:

res = bubble_client.get_objects(
        "Feedback",
        [
            BubbleField("source") == source_id,
            ],
    )
pd.DataFrame(res)

# Visualisation

In [ ]:
insight_layers = [
    pd.DataFrame(bubble_client.get_objects(
        "python_insight",
        [
            BubbleField("step") == i+1,
            BubbleField("company") == company_id,
            ],
    )) for i in range(n_layers)
]

In [ ]:
insight_layers[0].tail()

In [ ]:
sentences = insight_layers[0]["content"]
sentence_embeddings = embedding_model.encode(sentences)
sentence_embeddings.shape

In [ ]:
insight_layers[0]['parent']

In [ ]:
insight_layers[0]

In [ ]:
def to_int(i):
    try:
        return int(i)
    except:
        return -1

for layer in insight_layers:
    layer['parent'] = layer['parent'].apply(to_int)


In [ ]:
list(insight_layers[1]["content"])

In [ ]:
for i, layer in enumerate(insight_layers):
    print(list(insight_layers[0][insight_layers[0]['parent'] == 'None']["content"]))

In [ ]:
sum(insight_layers[0]['parent']<0)

In [ ]:
insight_layers[1].iloc[insight_layers[0]['parent'], "content"]

In [ ]:
insight_layers[0].loc[0, "cluster"] == 0

In [ ]:
map_to_parent(0, insight_layers[1])

In [ ]:
insight_layers[1].loc[0, 'parent']

In [ ]:
#@Insight Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=15)
umap_embeds = reducer.fit_transform(sentence_embeddings)

def map_to_parent(i, parents_df):
    try:
        return parents_df.loc[i, 'content']
    except:
        return ""
    
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={
    'content': insight_layers[0]['content'], 
    'parent': insight_layers[0]['parent'].apply(lambda x: map_to_parent(x, insight_layers[1])),
    'cluster': insight_layers[0]['cluster'].astype(str),
    })
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]
df_explore


In [ ]:

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    color='cluster',
    tooltip=['content', "parent"]
).properties(
    width=700,
    height=400
)
chart.interactive()

TF-IDF

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')

In [ ]:
def td_idf(documents)
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(documents)
    feature_names = vectorizer.get_feature_names_out()
    dense = vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    df = df[df.columns.difference(stopwords.words('french'))]


In [ ]:
df = td_idf(feedbacks_df['content'])
#print('\n'.join(df.columns))

In [ ]:
#print('\n'.join(df.columns))

In [ ]:

def get_top_two_columns(row):
    top_two_indexes = row.nlargest(5).index.tolist()
    return top_two_indexes

top_two_columns_df = df.apply(get_top_two_columns, axis=1)

print(top_two_columns_df)

In [ ]:
#print('\n'.join(insights_df['content']))